In [ ]:
# Import libraries
from datetime import datetime
import glob
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import xskillscore as xs
import hvplot.xarray
import metpy.calc as mpcalc
import metpy.plots as mpplots
from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays
from siphon.catalog import TDSCatalog
import re
import dask
import gc
from dask.distributed import Client

In [29]:
from datetime import datetime

In [31]:
# Axis extents for each region in [W, E, S, N] format
# Timezones in hours +- GMT
settings = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": {"extent": [-91, -81, 7, 17], "tz": -6},
    # South America (mostly eastern Brazil)
    "sa": {"extent": [-65, -30, -15, 0], "tz": -3},
    # Western Australia (mostly near the west coast)
    "wa": {"extent": [113, 123, -35, -30], "tz": +8}
}

In [32]:
# months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
#           "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
# years = range(1981, 2022)
avhrr_earliest = "Jan-1981"
modis_earliest = "Mar-2000"
avhrr_latest = "Dec-2018"
modis_latest = "Dec-2021"
subsets = ["all", "DJF", "MAM", "JJA", "SON",
           "Jan", "Feb", "Mar", "Apr", "May", "Jun",
           "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [33]:
def glass_data_source_to_use(periodstart, periodend):
    """
    Select which GLASS data source (AVHRR or MODIS) to use
    
    Arguments:
        periodstart (datetime.datetime): Start of period coverage
        periodend (datetime.datetime): End of period coverage
        
    Returns:
        data_soure (str): String indicating whether to use
                        "avhrr" or "modis" data for the given period
    
    For the given period, select the most appropriate GLASS data source
    to use (out of AVHRR and MODIS). MODIS is preferentially selected
    where the given period is completely contained within the time range
    of MODIS data. Otherwise, AVHRR data is used. Periods which simultaneously
    cover both an AVHRR-only period (i.e. before Mar-2000) and a MODIS-only 
    period (i.e. after Dec-2018) cannot be selected since summary statistics
    over this data is subject to artefacts from the change in instruments.
    """
    if ((periodstart >= datetime.strptime(avhrr_earliest, "%b-%Y")) & 
        (periodstart < datetime.strptime(modis_earliest, "%b-%Y")) &
        (periodend <= datetime.strptime(avhrr_latest, "%b-%Y"))):
        data_source = "avhrr"
    elif ((periodstart >= datetime.strptime(modis_earliest, "%b-%Y")) &
          (periodend <= datetime.strptime(modis_latest, "%b-%Y"))):
        data_source = "modis"
    else:
        raise Exception("If periodstart is before Mar-2000, " +
                        "periodend cannot be after Dec-2018 " +
                        "(since this would cover both an " +
                        "AVHRR-only and a MODIS-only period)")
    return(data_source)

In [34]:
def calc_mlai_climatology(region, periodstart, periodend, subset, outputname=None):
    """
    Calculate mean leaf area index (MLAI) climatology
    
    Arguments:
        region (str): Region to perform calculation over.
                        Must be one of ["ca", "sa", "wa"]
        periodstart (str): Start of period to perform calculation over.
                        Must be of form "%b-%Y" eg. "Jul-1990".
                        Must be between "Jan-1981" and "Dec-2021".
        periodend (str): End of period to perform calculation over.
                        Must be of form "%b-%Y" eg. "Jul-1990".
                        Must be between "Jan-1981" and "Dec-2021".
        subset (str): Subset of period to perform calculation over.
                        Must be one of ["all", "DJF", "MAM", "JJA", "SON",
                        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
                        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
        outputname (str): Name of output netcdf4 file. If none is
                        given then the default will be
                        region_periodstart_periodend_subset_mlai.nc
                        
    Returns:
        outputname (.nc file): Netcdf4 file in data_processed folder
                        describing MLAI climatology
    
    For each grid cell, calculate the mean leaf area index (MLAI) over 
    the period from periodstart to periodend. The calculation uses 8-day 
    satellite HDF data from the data_raw folder as input, then outputs 
    the result as a netcdf4 file into the data_processed folder. MODIS data
    is preferentially used where the given period is completely contained 
    within the time range of MODIS data. Otherwise, AVHRR data is used.
    """
    assert region in settings.keys(), \
        f"region must be one of: {*[*settings],}"
    # assert(isinstance(periodstart, str)), {"periodstart must be a string " +
    #                                       "of form '%b-%Y' eg. Jul-1990"}
    # assert(len(periodstart)==8), {"periodstart must be a string " +
    #                               "of form '%b-%Y' eg. Jul-1990"}
    # assert(periodstart[3]=="-"), {"periodstart must be a string " +
    #                               "of form '%b-%Y' eg. Jul-1990"}
    # assert(periodstart[0:3] in months), {"periodstart must be a string " +
    #                                      "of form '%b-%Y' eg. Jul-1990"}
    # assert(int(periodstart[4:8]) in years), {"periodstart must be a string " +
    #                                          "of form '%b-%Y' eg. Jul-1990"}
    periodstart = datetime.strptime(periodstart, "%b-%Y")
    periodend = datetime.strptime(periodend, "%b-%Y")
    assert periodend >= periodstart, \
        "periodend must be equal to or later than periodstart"
    assert periodstart >= datetime.strptime(avhrr_earliest, "%b-%Y"), \
        f"periodstart must be equal to or later than {avhrr_earliest}"
    assert periodend <= datetime.strptime(modis_latest, "%b-%Y"), \
        f"periodend must be equal to or earlier than {modis_latest}"
    data_source = glass_data_source_to_use(periodstart, periodend)
    assert subset in subsets, \
        f"subset must be one of: {*[*subsets],}"
    if outputname:
        assert(isinstance(outputname, str)), \
            "outputname must be a character string or None (default)"
    else:
        outputname = ("{}_{}_{}_{}_mlai_{}.nc"
                      .format(region, periodstart.strftime("%b-%Y"),
                              periodend.strftime("%b-%Y"), subset, data_source))
    
    # assert all file and there and data_download notebook was run properly
    
    
    print(region)
    print(data_source)
    print(periodstart)
    print(periodend)
    print(subset)
    print(outputname)

In [89]:
calc_mlai_climatology("ca", "Jul-2000", "Aug-2021", "JJA")

ca
modis
2000-07-01 00:00:00
2021-08-01 00:00:00
JJA
ca_Jul-2000_Aug-2021_JJA_mlai_modis.nc


In [80]:
calc_mlai_climatology?

Signature:
calc_mlai_climatology(
    region,
    periodstart,
    periodend,
    subset,
    outputname=None,
)
Docstring:
Calculate mean leaf area index (MLAI) climatology

Arguments:
    region (str): Region to perform calculation over.
                    Must be one of ["ca", "sa", "wa"]
    periodstart (str): Start of period to perform calculation over.
                    Must be of form "%b-%Y" eg. "Jul-1990".
                    Must be between "Jan-1981" and "Dec-2021".
    periodend (str): End of period to perform calculation over.
                    Must be of form "%b-%Y" eg. "Jul-1990".
                    Must be between "Jan-1981" and "Dec-2021".
    subset (str): Subset of period to perform calculation over.
                    Must be one of ["all", "DJF", "MAM", "JJA", "SON",
                    "Jan", "Feb", "Mar", "Apr", "May", "Jun",
                    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    outputname (str): Name of output netcdf4 file. If none is
       

In [13]:
import xarray as xr
from glob import glob

In [21]:
files = glob("../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981*")

In [23]:
files.sort()

In [24]:
files

['../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-001.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-009.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-017.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-025.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-033.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-041.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-049.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-057.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-065.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-073.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-081.hdf',
 '../data_raw/global_glass-lai-avhrr_8-day/

In [68]:
def preprocess_glass(ds):
    time = ds.encoding["source"][-12:-4]
    time = datetime.strptime(time, "%Y-%j")
    return(ds.expand_dims({"time": [time]}))

In [105]:
dset = xr.open_mfdataset(files, engine = "rasterio", preprocess=preprocess_glass).chunk(chunks = {"time": 10})
dset["band_data"]

<xarray.DataArray 'band_data' (time: 46, band: 1, y: 3600, x: 7200)>
dask.array<rechunk-merge, shape=(46, 1, 3600, 7200), dtype=float32, chunksize=(10, 1, 3600, 7200), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1981-01-01 1981-01-09 ... 1981-12-27
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 0
Attributes:
    long_name:  Leaf Area Index 8-Day 0.05-degrees GEO Grid
    units:      m^2/m^2

In [90]:
dset = xr.open_mfdataset(files, engine = "rasterio", preprocess=preprocess_glass)
dset["band_data"].chunk(chunks = 100)

,Array,Chunk
Bytes,4.44 GiB,1.75 MiB
Shape,"(46, 1, 3600, 7200)","(46, 1, 100, 100)"
Count,6724 Tasks,2592 Chunks
Type,float32,numpy.ndarray


In [89]:
dset = xr.open_mfdataset(files, engine = "rasterio", preprocess=preprocess_glass)
dset["band_data"].chunk(chunks = 10)

,Array,Chunk
Bytes,4.44 GiB,3.91 kiB
Shape,"(46, 1, 3600, 7200)","(10, 1, 10, 10)"
Count,2592796 Tasks,1296000 Chunks
Type,float32,numpy.ndarray


In [40]:
ds_lai = xr.open_dataset("../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_1981-001.hdf", 
                           chunks = {'time': '500MB'},
                           engine = "rasterio")
ds_lai

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 dask.array<chunksize=(1, 3600, 7200), meta=np.ndarray>

In [41]:
ds_lai2 = xr.open_dataset("../data_raw/global_glass-lai-avhrr_8-day/global_glass-lai-avhrr_8-day_2018-361.hdf", 
                           engine = "rasterio")
ds_lai2

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 ...

In [42]:
ds_lai3 = xr.open_dataset("../data_raw/global_glass-lai-modis_8-day/global_glass-lai-modis_8-day_2018-361.hdf", 
                           engine = "rasterio")
ds_lai3

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 ...

In [45]:
preprocess(ds_lai3)

'../data_raw/global_glass-lai-modis_8-day/global_glass-lai-modis_8-day_2018-361.hdf'

In [47]:
preprocess(ds_lai3)[-12:-4]

'2018-361'

In [66]:
ds_lai3.expand_dims("time")

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600, time: 1)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 0
Dimensions without coordinates: time
Data variables:
    band_data    (time, band, y, x) float32 nan nan nan nan ... nan nan nan nan

In [67]:
x = datetime.strptime("1997-365", "%Y-%j")
ds_lai3.expand_dims({"time": [x]})

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 1997-12-31
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 0
Data variables:
    band_data    (time, band, y, x) float32 nan nan nan nan ... nan nan nan nan

In [50]:
ds_lai3.assign_coords({"time": "test"})

<xarray.Dataset>
Dimensions:      (band: 1, x: 7200, y: 3600)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 0
    time         <U4 'test'
Data variables:
    band_data    (band, y, x) float32 nan nan nan nan nan ... nan nan nan nan

In [35]:
datetime.strptime(avhrr_earliest, "%b-%Y").strftime("%Y-%j")

'1981-001'

In [36]:
datetime.strptime("1997-365", "%Y-%j").strftime("%b-%Y")

'Dec-1997'

In [37]:
datetime.strptime("1997-365", "%Y-%j")

datetime.datetime(1997, 12, 31, 0, 0)

In [76]:
type(datetime.strptime(avhrr_earliest, "%b-%Y"))

datetime.datetime

In [33]:
datetime.strptime(avhrr_earliest, "%b-%Y").strftime("%b-%Y")

'Jan-1981'

In [87]:
datetime.strptime("Feb-1980", "%b-%Y") >= datetime.strptime("Feb-1980", "%b-%Y")

True

In [142]:
True & False

False

In [187]:
isinstance(None, str)

False

In [194]:
if None:
    print("hello")
else:
    print("bye")

bye


In [10]:
areas["wa"]["extent"]

[113, 123, -35, -30]

In [ ]:
# Axis extents for each region in [W, E, S, N] format
extent = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": [-91, -81, 7, 17],
    # South America (mostly eastern Brazil)
    "sa": [-65, -30, -15, 0],
    # Western Australia (mostly near the west coast)
    "wa": [113, 123, -35, -30]
}

In [ ]:
tz = {"ca": -6, # +- 0
      "sa": -3, # +- 1.5
      "wa": +8} # +- 0